## 4 MNIST Dataset Clustering

This code borrows heavily from code written for homework 14 for Georgia Tech Class CSE6040, which I took in Fall 2023.

#### Load Dataset
To begin, we read in the datasets and import packages

In [1]:
from PIL import Image
from matplotlib.pyplot import imshow
%matplotlib inline

import scipy.io
import numpy as np
import math
import pandas as pd
import statistics

## load matlab file
mat = scipy.io.loadmat('mnist_10digits.mat')

## import files
xtrain = mat.get('xtrain') / 255
ytrain = mat.get('ytrain')


#### Initialize Centers
To begin k-means, random centers must be chosen. The below funtion initializes these random centers based on the shape of the image array.

In [2]:
def init_centers(X, k):
    """
    Randomly samples k observations from X as centers.
    Returns these centers as a (k x d) numpy array.
    """
    r, c = X.shape
    samples = np.random.choice(r, size=k, replace=False)

    return X[samples, :]

#### Compute Distance
The below function computes the Euclidian distance between point $x_i$ to center $\mu_i$.
We initialize a numpy array to hold the distances based on our image shape and matrix shape, and fill it with the Euclidian distances.

#### Compute Distance - Euclidean
The below function computes the Euclidean distance between point $x_i$ to center $\mu_i$.
We initialize a numpy array to hold the distances based on our image shape and matrix shape, and fill it with the Euclidian distances.

In [3]:
def compute_d2(X, centers):
    m, _ = X.shape
    n, _ = centers.shape
    
    S = np.empty((m, n))
    
    for i in range(m):
        S[i,:] = np.linalg.norm(X[i,:] - centers, ord=2, axis=1)**2
        
    return S

#### Compute Distance - Manhattan
The below function computes the Manhattan distance between point $x_i$ to center $\mu_i$.
We initialize a numpy array to hold the distances based on our image shape and matrix shape, and fill it with the Manhattan distances.

In [4]:
def compute_dm(X, centers):
    m, _ = X.shape
    n, _ = centers.shape
    
    S = np.empty((m, n))
    
    for i in range(m):
        S[i,:] = np.linalg.norm((X[i,:] - centers), ord=1, axis = 1)
        
    return S



#### Assign Cluster Label
The below function uses the matrix from above to assign a cluster label to each point $x_i$.
A single vector is returned, $y_i$, the cluster the point $x_i$ is assigned to from using $\arg\min$

In [5]:
def assign_cluster_labels(S):
    ### BEGIN SOLUTION
    return np.argmin(S, axis=1)
    ### END SOLUTION

# Cluster labels:     0    1
S_test1 = np.array([[0.3, 0.2],  # --> cluster 1
                    [0.1, 0.5],  # --> cluster 0
                    [0.4, 0.2]]) # --> cluster 1
y_test1 = assign_cluster_labels(S_test1)
print("You found:", y_test1)



You found: [1 0 1]


#### Update Cluster Centers
Now we have cluster labels and the points inside. From here, we update the cluster center based on this information. The input for the function is the image and the label from the previous function. The new center is the mean of the points inside the cluster.

In [6]:
def update_centers(X, y):
    # X[:m, :d] == m points, each of dimension d
    # y[:m] == cluster labels
    m, d = X.shape
    k = max(y) + 1
    assert m == len(y)
    assert (min(y) >= 0)
    
    centers = np.empty((k, d))
    for j in range(k):
        # Compute the new center of cluster j,
        # i.e., centers[j, :d].
        ### BEGIN SOLUTION
        centers[j, :d] = np.mean(X[y == j, :], axis=0)
        ### END SOLUTION
    return centers

#### Within-Cluster Sum of Squares
This metric tells us our sum of squares, which is the value to be minimized. The smaller the value, the more compact the clusters are, thus better assigned.

In [7]:
def WCSS(S):
    ### BEGIN SOLUTION
    return np.sum(np.amin(S, axis=1))
    ### END SOLUTION

#### K-means
The first function is chekcing if the centers have converged.
The second function is the calculation of kmeans:
1. initalize centers either by entering the values or by the init_centers function from before.
2. Set that the centers have not yet converged
3. while loop that computes distances and updates centers until they converge

Two functions are run, once for the Euclidean distance and once for the Manhattan Distance. The functions return the group and the centers of the group

In [8]:
def has_converged(old_centers, centers):
    return set([tuple(x) for x in old_centers]) == set([tuple(x) for x in centers])

def kmeans(X, k,
           starting_centers=None,
           max_steps=np.inf):
    if starting_centers is None:
        centers = init_centers(X, k)
    else:
        centers = starting_centers
        
    converged = False
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_centers = centers
        ### BEGIN SOLUTION
        S = compute_d2(X, centers)
        labels = assign_cluster_labels(S)
        centers = update_centers(X, labels)
        converged = has_converged(old_centers, centers)
        ### END SOLUTION

    return labels, centers

def kmeans_manh(X, k,
           starting_centers=None,
           max_steps=np.inf):
    if starting_centers is None:
        centers = init_centers(X, k)
    else:
        centers = starting_centers
        
    converged = False
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_centers = centers
        ### BEGIN SOLUTION
        S = compute_dm(X, centers)
        labels = assign_cluster_labels(S)
        centers = update_centers(X, labels)
        converged = has_converged(old_centers, centers)
        ### END SOLUTION

    return labels, centers

#### Build models for each k-means

In [9]:
labels_l2 = kmeans(xtrain, 10)
labels_m = kmeans_manh(xtrain, 10)

#### Calculating Purity Score
The k-means function above groups points into clusters. For each data point, the answer is in the data set ytrain.
1. Assign label outputs to the nodes, which are the unique values 0-9, the digits.
2. Within the group, calculate the most frequent occurence.
3. Calculate purity score as ratio of most frequence occurence to number in group.


In [10]:
### For L2 Square Distance
nodes = np.unique(labels_l2[0])

centers = labels_l2[1]
label = labels_l2[0]
nodes_purity = pd.DataFrame(columns = ["group", "score"]) ## establish dataframe

for n in nodes:
    grouplabel = label[(ytrain[0]==n)] ## the group according to training
    
    # Program to find most frequent element in a list
    # approach 7 https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/
    def most_frequent(List):
        unique, counts = np.unique(List, return_counts=True)
        index = np.argmax(counts)
        return unique[index]

    freq = (most_frequent(grouplabel)) ## find most frequent for group type
    purity = sum(grouplabel==freq) / len(grouplabel) ## calculate purity score
    

    nodes_purity.loc[len(nodes_purity.index)] = [n.astype(int), purity]

display(nodes_purity)

print(nodes_purity.to_latex())

,group,score
0,0.0,0.787945
1,1.0,0.551765
2,2.0,0.638134
3,3.0,0.610178
4,4.0,0.344574
5,5.0,0.321527
6,6.0,0.828320
7,7.0,0.513807
8,8.0,0.505213
9,9.0,0.443100


\begin{tabular}{lrr}
\toprule
 & group & score \\
\midrule
0 & 0.000000 & 0.787945 \\
1 & 1.000000 & 0.551765 \\
2 & 2.000000 & 0.638134 \\
3 & 3.000000 & 0.610178 \\
4 & 4.000000 & 0.344574 \\
5 & 5.000000 & 0.321527 \\
6 & 6.000000 & 0.828320 \\
7 & 7.000000 & 0.513807 \\
8 & 8.000000 & 0.505213 \\
9 & 9.000000 & 0.443100 \\
\bottomrule
\end{tabular}



In [14]:
### For Manhattan Distance

nodes_m = np.unique(labels_m[0])

centers_m = labels_m[1]
label_m = labels_m[0]
nodes_puritym = pd.DataFrame(columns = ["group", "score"]) ## establish dataframe

for n in nodes_m:
    grouplabel = label_m[(ytrain[0]==n)] ## the grouop according to training
    
    # Program to find most frequent
    # element in a list
    # approach 7 https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/
    def most_frequent(List):
        unique, counts = np.unique(List, return_counts=True)
        index = np.argmax(counts)
        return unique[index]

    freq = (most_frequent(grouplabel)) ## find most frequent for group type
    purity = sum(grouplabel==freq) / len(grouplabel) ## calculate purity score
    

    nodes_puritym.loc[len(nodes_puritym.index)] = [n.astype(int), purity]
    
display(nodes_puritym)
print(nodes_puritym.to_latex())

,group,score
0,0.0,0.423096
1,1.0,0.997330
2,2.0,0.440584
3,3.0,0.426358
4,4.0,0.428278
5,5.0,0.352702
6,6.0,0.608314
7,7.0,0.676616
8,8.0,0.402666
9,9.0,0.506472


\begin{tabular}{lrr}
\toprule
 & group & score \\
\midrule
0 & 0.000000 & 0.423096 \\
1 & 1.000000 & 0.997330 \\
2 & 2.000000 & 0.440584 \\
3 & 3.000000 & 0.426358 \\
4 & 4.000000 & 0.428278 \\
5 & 5.000000 & 0.352702 \\
6 & 6.000000 & 0.608314 \\
7 & 7.000000 & 0.676616 \\
8 & 8.000000 & 0.402666 \\
9 & 9.000000 & 0.506472 \\
\bottomrule
\end{tabular}



In [15]:
print(f'''Mean Purity Score for Euclidean Distance: {round(statistics.mean(nodes_purity["score"]), 3)}
Minimum Purity Score for Euclidean Distance: {round(min(nodes_purity["score"]), 3)}
Maximum Purity Score for Euclidean Distance: {round(max(nodes_purity["score"]), 3)}

Mean Purity Score for Manhattan Distance: {round(statistics.mean(nodes_puritym["score"]), 3)}
Minimum Purity Score for Manhattan Distance: {round(min(nodes_puritym["score"]), 3)}
Maximum Purity Score for Manhattan Distance: {round(max(nodes_puritym["score"]), 3)}''')

Mean Purity Score for Euclidean Distance: 0.554
Minimum Purity Score for Euclidean Distance: 0.322
Maximum Purity Score for Euclidean Distance: 0.828

Mean Purity Score for Manhattan Distance: 0.526
Minimum Purity Score for Manhattan Distance: 0.353
Maximum Purity Score for Manhattan Distance: 0.997
